# Orientation Tuning — Overlay of Filtered Units
This view overlays **all filtered units**. Sliders are debounced for speed in the browser.


In [ ]:
import numpy as np, pandas as pd
import ipywidgets as W
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# ---------- Load data (content/data/...) ----------
meta   = pd.read_csv("data/meta.csv")                 # columns: unit_id, layer, OSI, selectivity
npz    = np.load("data/tuning_curves.npz")            # arrays: angles (A,), curves (N,A)
angles = npz["angles"]
curves = npz["curves"]

N, A = curves.shape
layer_opts = tuple(sorted(meta["layer"].unique().tolist()))

# ---------- Pre-compute normalizations for speed ----------
curves_raw  = curves.copy()
mu  = curves.mean(axis=1, keepdims=True)
std = curves.std(axis=1, keepdims=True) + 1e-12
curves_z    = (curves - mu) / std
peak = np.maximum(1e-12, np.max(np.abs(curves), axis=1, keepdims=True))
curves_peak = curves / peak

norm_map = {"raw": curves_raw, "z": curves_z, "peak": curves_peak}

# ---------- Controls (debounced) ----------
layers = W.SelectMultiple(options=layer_opts, value=layer_opts, description="Layers", rows=3, layout=W.Layout(width="220px"))
osi    = W.FloatRangeSlider(value=(0.0, 1.0), min=0.0, max=1.0, step=0.01, description="OSI",
                            readout_format=".2f", layout=W.Layout(width="380px"), continuous_update=False)
sel    = W.FloatRangeSlider(value=(0.0, 1.0), min=0.0, max=1.0, step=0.01, description="Select.",
                            readout_format=".2f", layout=W.Layout(width="380px"), continuous_update=False)
norm   = W.Dropdown(options=[("Raw","raw"), ("Z-score per unit","z"), ("Peak normalize (max=1)","peak")],
                    value="raw", description="Normalize")
focus  = W.Dropdown(options=[(f"Unit {u}", int(u)) for u in meta["unit_id"].tolist()],
                    value=int(meta["unit_id"].iloc[0]), description="Focus unit")
status = W.HTML()
plot_out = W.Output(layout=W.Layout(border="1px solid #e5e5e5", min_height="420px", width="100%"))

def filtered_idx():
    sel_layers = list(layers.value)
    if len(sel_layers) == 0:
        return np.array([], dtype=int)
    m = (
        meta["layer"].isin(sel_layers)
        & (meta["OSI"] >= osi.value[0]) & (meta["OSI"] <= osi.value[1])
        & (meta["selectivity"] >= sel.value[0]) & (meta["selectivity"] <= sel.value[1])
    )
    return np.flatnonzero(m.values)

def update_plot(*_):
    idx = filtered_idx()
    status.value = (f"<b>Matched units:</b> {idx.size} | Layers: {', '.join(layers.value)} | "
                    f"OSI: {osi.value[0]:.2f}–{osi.value[1]:.2f} | Select.: {sel.value[0]:.2f}–{sel.value[1]:.2f}")
    Ybank = norm_map[norm.value]

    with plot_out:
        clear_output(wait=True)
        fig = plt.figure(figsize=(7.5, 4.2), dpi=140)
        ax  = plt.gca()
        if idx.size == 0:
            ax.text(0.5, 0.5, "No units match the filter", ha="center", va="center", transform=ax.transAxes)
            ax.set_axis_off()
            plt.show()
            return

        # --- Overlay all filtered units ---
        # Use one vectorized draw call when possible
        for ii in idx:
            ax.plot(angles, Ybank[ii], lw=1.0, alpha=0.30)

        # --- Focus unit (always draw on top if present) ---
        if focus.value in meta["unit_id"].values:
            row = int(meta.index[meta["unit_id"] == focus.value][0])
            ax.plot(angles, Ybank[row], lw=2.6)

        ax.set_xlabel("Orientation (deg)")
        ax.set_ylabel("Response (a.u.)" if norm.value == "raw" else f"Response ({norm.value})")
        ax.set_title("Orientation tuning (overlay of filtered units)")
        ax.grid(True, alpha=0.25)
        plt.tight_layout()
        plt.show()

# Wire callbacks (debounced sliders only fire on release)
for w in (layers, osi, sel, norm, focus):
    w.observe(update_plot, names="value")

# Initial render
update_plot()

controls = W.HBox([W.VBox([layers, norm]), W.VBox([osi, sel, focus])])
display(controls, status, plot_out)
